In [2]:
import nltk
from sklearn.metrics import classification_report, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import torch
import torch.nn as nn

#from torch_model_base import TorchModelBase
#from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_rnn_classifier import TorchRNNDataset, TorchRNNClassifier, TorchRNNModel
import utils

import pandas as pd
from collections import Counter

In [3]:
# refresh torch rnn classifier:
import importlib
import torch_rnn_classifier
importlib.reload(torch_rnn_classifier)
from torch_rnn_classifier import TorchRNNDataset

In [4]:
import json

In [5]:
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]

In [6]:
# now get data into format that TorchRNN expects:
X=[] 
y=[]
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    auxX = []
    auxy = []
    if annot[j]['spans']!=[]: # are there annot for this example?
        for i in range(0,len(a)):
            #token_element = (a[i]['text'],a[i]['label'])
            auxX.append(a[i]['text'])
            auxy.append(a[i]['label'])
        X.append(auxX)
        y.append(auxy)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = X[:120], X[120:], y[:120], y[120:]
vocab = sorted({w for seq in X_train for w in seq}) + ["$UNK"]

In [7]:
# reload vsm module
import torch_rnn_classifier, torch_model_base
import importlib
importlib.reload(torch_model_base)
importlib.reload(torch_rnn_classifier)
from torch_model_base import TorchModelBase
from torch_rnn_classifier import TorchRNNClassifier, TorchRNNModel

In [8]:
class TorchRNNSequenceLabeler(TorchRNNClassifier):

    def build_graph(self): # uses this build_graph instead of TorchRNNClassifier.build_graph
        print("here0")
        rnn = TorchRNNModel(
            vocab_size=len(self.vocab),
            embedding=self.embedding,
            use_embedding=self.use_embedding,
            embed_dim=self.embed_dim,
            rnn_cell_class=self.rnn_cell_class,
            hidden_dim=self.hidden_dim,
            bidirectional=self.bidirectional,
            freeze_embedding=self.freeze_embedding)
        print("here02")
        model = TorchSequenceLabeler( # this defines self.model
            rnn=rnn,
            output_dim=self.n_classes_)
        self.embed_dim = rnn.embed_dim
        return model

    def build_dataset(self, X, y=None):
        START_TAG = "<START>"
        STOP_TAG = "<STOP>"
        X, seq_lengths = self._prepare_sequences(X) # converts tokens into tokenIds
        if y is None:
            return TorchRNNDataset(X, seq_lengths)
        else:
            # These are the changes from a regular classifier. All
            # concern the fact that our labels are sequences of labels.
            self.classes_ = sorted({x for seq in y for x in seq})
            self.classes_.append(START_TAG) # add start and stop tags
            self.classes_.append(STOP_TAG)
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            # `y` is a list of tensors of different length. Our Dataset
            # class will turn it into a padding tensor for processing.
            y = [torch.tensor([class2index[label] for label in seq])
                 for seq in y] # converts labels to indices
            return TorchRNNDataset(X, seq_lengths, y)

    def predict_proba(self, X):
        seq_lengths = [len(ex) for ex in X]
        # The base class does the heavy lifting:
        preds = self._predict(X)
        # Trim to the actual sequence lengths:
        preds = [p[: l] for p, l in zip(preds, seq_lengths)]
        # Use `softmax`; the model doesn't do this because the loss
        # function does it internally.
        probs = [torch.softmax(seq, dim=1) for seq in preds]
        return probs

    def predict(self, X):
        probs = self.predict_proba(X)
        return [[self.classes_[i] for i in seq.argmax(axis=1)] for seq in probs] # seq.argmax(axis=1) gives index of col that maximizes softmax prob
        # see difference vs TorchRNNClassifier.predict

    def score(self, X, y):
        preds = self.predict(X)
        flat_preds = [x for seq in preds for x in seq]
        flat_y = [x for seq in y for x in seq]
        return utils.safe_macro_f1(flat_y, flat_preds)
    

    
    
class TorchSequenceLabeler(nn.Module): # no self.hidden_layer or self.classifier_activation as TorchRNNClassifierModel
    def __init__(self, rnn, output_dim):
        print("here021")
        super().__init__()
        self.rnn = rnn
        self.output_dim = output_dim
        if self.rnn.bidirectional:
            self.classifier_dim = self.rnn.hidden_dim * 2
        else:
            self.classifier_dim = self.rnn.hidden_dim
        self.classifier_layer = nn.Linear(
            self.classifier_dim, self.output_dim)

    def forward(self, X, seq_lengths): # X is (noExsInBatch,MaxLen)=(108,117), seq_lengths is the number of tokens in each example in each batch
        # this is the forward method of self.model
        print("here2")
        outputs, state = self.rnn(X, seq_lengths) # X is (batchSize, maxLen of exs in batch); outputs is (noTokensInEx,hiddDim), state is ((batch_size,1,hiddDim),(batch_size,1,hiddDim)) = (finalHiddState,finalCellState) 
        #print(outputs.data.shape)
        #print(state[0].data.shape)
        #print(state[1].data.shape)
        outputs, seq_length = torch.nn.utils.rnn.pad_packed_sequence(
            outputs, batch_first=True) # outputs is (batchSize,MaxLen of examples in batch,hidden_dim); seq_length is noTokenInEx for each ex in batch
        #print(outputs.data.shape)
        #print(seq_length)
        logits = self.classifier_layer(outputs) # this is an FCL from hidden_dim to output_dim (NoLabelClasses)
        # logits are (108,117,12) or (1,11,5) = (batchSize,MaxLen of examples in batch,noLabelClasses) noLabelClasses include Start + End
        # During training, we need to swap the dimensions of logits
        # to accommodate `nn.CrossEntropyLoss`:
        print(logits)
        if self.training:
            return logits.transpose(1, 2) # transpose dimensions 1 and 2 w/ each other (3d array) # outputs (108,12,117) or (1,5,11)
        else:
            return logits

In [9]:
seq_mod = TorchRNNSequenceLabeler(
    vocab,
    early_stopping=True,
    eta=0.001)

In [10]:
X_train = ["the wall street journal reported today that apple corporation made money".split(),"georgia tech is a university in georgia".split()]
y_train = ["B I I I O O O B I O O".split(),"B I O O O O B".split()]
vocab = sorted({w for seq in X_train for w in seq}) + ["$UNK"]

In [24]:
pd.DataFrame(list(zip(X_train[0], y_train[0])), columns=["X", "y"])

,X,y
0,the,B
1,wall,I
2,street,I
3,journal,I
4,reported,O
5,today,O
6,that,O
7,apple,B
8,corporation,I
9,made,O


In [25]:
%time _ = seq_mod.fit(X_train, y_train)

Stopping after epoch 20. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 1.0853766202926636

here00
here0
here01
here02
here021
batch1
here2
here21
tensor([[[-0.0468,  0.1913, -0.0816, -0.0243, -0.0453],
         [-0.0522,  0.1927, -0.1355, -0.0581, -0.0249],
         [-0.0612,  0.1893, -0.1723, -0.0763, -0.0242],
         [-0.0678,  0.1864, -0.1940, -0.0844, -0.0285],
         [-0.0721,  0.1843, -0.2059, -0.0873, -0.0331],
         [-0.0746,  0.1827, -0.2124, -0.0880, -0.0368],
         [-0.0761,  0.1816, -0.2160, -0.0877, -0.0395],
         [-0.0770,  0.1808, -0.2178, -0.0872, -0.0414],
         [-0.0776,  0.1801, -0.2187, -0.0867, -0.0427],
         [-0.0779,  0.1796, -0.2192, -0.0862, -0.0435],
         [-0.0782,  0.1794, -0.2195, -0.0859, -0.0440]]], device='cuda:0',
       grad_fn=<AddBackward0>)
here2
here21
tensor([[[-0.0490,  0.2121, -0.0366, -0.0381, -0.0686],
         [-0.0529,  0.2263, -0.0614, -0.0797, -0.0595],
         [-0.0598,  0.2306, -0.0799, -0.1025, -0.0643],
         [-0.0649,  0.2325, -0.0902, -0.1135, -0.0713],
         [-0.0680,  0.2335, -0.0953, -0.11

In [13]:
seq_mod.predict("the wall street journal reported today that apple corporation made money".split())

here2
here21
tensor([[[ 0.0533, -0.0668,  0.0509, -0.1268, -0.0599],
         [ 0.0757, -0.0614,  0.1302, -0.1578, -0.0358],
         [ 0.0825, -0.0630,  0.1838, -0.1707, -0.0218],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920]],

        [[ 0.0533, -0.0668,  0.0509, -0.1268, -0.0599],
         [ 0.0757, -0.0614,  0.1302, -0.1578, -0.0358],
         [ 0.0825, -0.0630,  0.1838, -0.1707, -0.0218],
         [ 0.0855, -0.0644,  0.2189, -0.1760, -0.0134],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.0464, -0.0639, -0.0920],
         [-0.0329, -0.1122, -0.04

[['B', 'O', 'O'],
 ['B', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O'],
 ['B', 'O', 'O', 'O', 'O']]

In [592]:
from torchcrf import CRF

class TorchCRFSequenceLabeler(TorchRNNClassifier):

    def build_graph(self): # uses this build_graph instead of TorchRNNClassifier.build_graph
        print("here0")
        rnn = TorchRNNModel(
            vocab_size=len(self.vocab),
            embedding=self.embedding,
            use_embedding=self.use_embedding,
            embed_dim=self.embed_dim,
            rnn_cell_class=self.rnn_cell_class,
            hidden_dim=self.hidden_dim,
            bidirectional=self.bidirectional,
            freeze_embedding=self.freeze_embedding)
        print("here02")
        model = TorchSequenceLabeler( # this defines self.model
            rnn=rnn,
            output_dim=self.n_classes_)
        self.embed_dim = rnn.embed_dim
        return model

    def build_dataset(self, X, y=None):
        START_TAG = "<START>"
        STOP_TAG = "<STOP>"
        X, seq_lengths = self._prepare_sequences(X) # converts tokens into tokenIds
        if y is None:
            return TorchCRFDataset(X, seq_lengths)
            #return TorchRNNDataset(X, seq_lengths)
        else:
            # These are the changes from a regular classifier. All
            # concern the fact that our labels are sequences of labels.
            self.classes_ = sorted({x for seq in y for x in seq})
            self.classes_.append(START_TAG) # add start and stop tags
            self.classes_.append(STOP_TAG)
            self.n_classes_ = len(self.classes_)
            class2index = dict(zip(self.classes_, range(self.n_classes_)))
            print(class2index)
            # `y` is a list of tensors of different length. Our Dataset
            # class will turn it into a padding tensor for processing.
            y = [torch.tensor([class2index[label] for label in seq])
                 for seq in y] # converts labels to indices
            return TorchCRFDataset(X, seq_lengths, y)
            #return TorchRNNDataset(X, seq_lengths, y)

    def predict_proba(self, X):
        seq_lengths = [len(ex) for ex in X]
        # The base class does the heavy lifting:
        preds = self._predict(X)
        # Trim to the actual sequence lengths:
        preds = [p[: l] for p, l in zip(preds, seq_lengths)]
        # Use `softmax`; the model doesn't do this because the loss
        # function does it internally.
        probs = [torch.softmax(seq, dim=1) for seq in preds]
        return probs

    def predict(self, X):
        probs = self.predict_proba(X)
        return [[self.classes_[i] for i in seq.argmax(axis=1)] for seq in probs] # seq.argmax(axis=1) gives index of col that maximizes softmax prob
        # see difference vs TorchRNNClassifier.predict

    def score(self, X, y):
        preds = self.predict(X)
        flat_preds = [x for seq in preds for x in seq]
        flat_y = [x for seq in y for x in seq]
        return utils.safe_macro_f1(flat_y, flat_preds)


class TorchCRFDataset(TorchRNNDataset):

    @staticmethod
    def collate_fn(batch):
        """
        Format a batch of examples for use in both training and prediction.

        Parameters
        ----------
        batch : tuple of length 2 (prediction) or 3 (training)
            The first element is the list of input sequences. The
            second is the list of lengths for those sequences. The third,
            where present, is the list of labels.

        Returns
        -------
        X : torch.Tensor, shape `(batch_size, max_batch_length)`
            As padded by `torch.nn.utils.rnn.pad_sequence.

        seq_lengths : torch.LongTensor, shape `(batch_size, )`

        y : torch.LongTensor, shape `(batch_size, )`
            Only for training. In the case where `y` cannot be turned into
            a Tensor, we assume it is because it is a list of variable
            length sequences and to use `torch.nn.utils.rnn.pad_sequence`.
            The hope is that this will accomodate sequence prediction.

        """
        batch_elements = list(zip(*batch))
        X = batch_elements[0]
        seq_lengths = batch_elements[1]
        X = torch.nn.utils.rnn.pad_sequence(X, batch_first=True)
        seq_lengths = torch.tensor(seq_lengths)
        if len(batch_elements) == 3:
            y = batch_elements[2]
            # We can try to accommodate the case where `y` is a sequence
            # loss with potentially different lengths by resorting to
            # padding if creating a tensor is not possible:
            try:
                y = torch.tensor(y)
            # except ValueError:
            except TypeError:
                y = torch.nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=3) # need to pad with STOP tag
            return X, seq_lengths, y
        else:
            return X, seq_lengths


In [593]:
# Following converts words to indices and pads sequences
seq_mod = TorchCRFSequenceLabeler(
    vocab,
    early_stopping=True,
    eta=0.001)

In [ ]:
X_train = ["the wall street journal reported today that apple corporation made money".split(),"georgia tech is a university in georgia".split()]
y_train = ["B I I I O O O B I O O".split(),"B I O O O O B".split()]

In [594]:
torch.manual_seed(1)
dataset = seq_mod.build_dataset(X_train, y_train) # not good ... is padding with zeros (=B's)
dataloader = seq_mod._build_dataloader(dataset, shuffle=False) 
num_tags = 4
model = CRF(num_tags)
for batch_num, batch in enumerate(dataloader, start=1):
    x=batch[0]
    seq_length=batch[1]
    y=batch[2] 
    print(x)
    print(y)
    print(seq_length)
batch_size=2
emissions = torch.randn(batch_size, max(seq_length), num_tags)
print(emissions[0])
print(model(emissions,y)) # computes log likelihood
#model.decode(emissions)
a=model.transitions
print(a)
print(model.decode(emissions))

{'B': 0, 'I': 1, 'O': 2, '<START>': 3, '<STOP>': 4}
tensor([[13, 16, 10,  6,  9, 14, 12,  1,  2,  7,  8],
        [ 3, 11,  5,  0, 15,  4,  3,  0,  0,  0,  0]])
tensor([[0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2],
        [0, 1, 2, 2, 2, 2, 0, 3, 3, 3, 3]])
tensor([11,  7])
tensor([[ 1.7674, -0.0954,  0.1394, -1.5785],
        [-0.3206, -0.2993,  1.8793, -0.0721],
        [ 0.2753,  1.7163, -0.0561,  0.9107],
        [-1.3924,  2.6891, -0.1110,  0.2927],
        [ 2.0242, -0.0865,  0.0981, -1.2150],
        [ 0.7312,  1.1718, -0.9274,  0.5451],
        [ 0.2468,  1.1843, -0.7282,  1.1633],
        [-0.0091, -0.8425,  0.1374,  0.9386],
        [-1.8034, -1.3083,  0.4533,  1.1422],
        [ 0.2486, -1.7754, -0.0255, -1.0233],
        [ 0.1099, -0.6463,  0.4285,  1.4761]])
tensor(-39.0331, grad_fn=<SumBackward0>)
Parameter containing:
tensor([[ 0.0139, -0.0122,  0.0277,  0.0049],
        [ 0.0365, -0.0390, -0.0073, -0.0090],
        [ 0.0145, -0.0004,  0.0874,  0.0311],
        [-0.0372, -0.0604, 

In [525]:
tags = a.sequences
#print(tags.shape)
model(emissions,tags)

AttributeError: 'list' object has no attribute 'shape'

In [377]:
# note: this isn't exactly correct as training examples are shuffled (esp. max len of the smaller 12 ex batch is != 92)
auxMax=0
x_max_idx=108
for i in range(0,min(len(X_train),x_max_idx)):
    if len(X_train[i])>auxMax:
        auxMax=len(X_train[i])
print(auxMax)
auxMax2=0
x_min_idx=109
for i in range(max(0,x_min_idx),len(X_train)):
    if len(X_train[i])>auxMax2:
        auxMax2=len(X_train[i])
print(auxMax2)

117
92


In [107]:
y_pred = seq_mod.predict(X_test)
print(y_test[0])
print(y_pred[0])

['ORT', 'O', 'O', 'STRASSE', 'STRASSE', 'O', 'ORT', 'O', 'O', 'FLAECHE', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'O', 'O', 'QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERTRAG', 'DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O']
['KAEUFER', 'KAEUFER', 'O', 'O', 'KAEUFER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [108]:
labels=seq_mod.classes_
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

In [109]:
# unfold all our data - NOTE: this means we don't care about per sentence results. 
# i.e. each classification is worth same regardless of sentence in which it occurs
y_test_unfold = [y for element in y_test for y in element]
y_pred_unfold = [y for element in y_pred for y in element]

In [126]:
print(y_test_unfold[:10])
print(y_pred_unfold[:10])

['ORT', 'O', 'O', 'STRASSE', 'STRASSE', 'O', 'ORT', 'O', 'O', 'FLAECHE']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [82]:
# convert y_test and y_pred into binary formats
#from sklearn.preprocessing import MultiLabelBinarizer

In [110]:
print(classification_report(
    y_test_unfold, y_pred_unfold, labels=sorted_labels, digits=3
))

                     precision    recall  f1-score   support

                  O      0.808     0.888     0.846       643
            KAEUFER      0.070     0.278     0.112        18
DATUM_VERBUECHERUNG      0.000     0.000     0.000        25
      DATUM_VERTRAG      1.000     0.037     0.071        27
         VERKAEUFER      0.333     0.042     0.074        24
   TERRASSENGROESSE      0.000     0.000     0.000         5
        GESAMTPREIS      0.000     0.000     0.000        11
            FLAECHE      0.000     0.000     0.000        15
           IMMO_TYP      0.000     0.000     0.000        19
            QMPREIS      0.000     0.000     0.000        10
                ORT      0.000     0.000     0.000        26
            STRASSE      0.118     0.125     0.121        16

           accuracy                          0.691       839
          macro avg      0.194     0.114     0.102       839
       weighted avg      0.664     0.691     0.657       839



Now try with leading "B-" and "I-"

In [83]:
########## ONLY RUN IF WE WANT TO ADD LEADING "B-" / "I-" TO CLASS LABEL
# now use above code and loop through all items of annot list:
# addLeading=1 for "Yes" (i.e. add leading "B-","I-" to annot); 0 for "No" (i.e. add labels to annot simply as they are)
addLeading = 1

if addLeading == 1:
    for j in range(0,len(annot)):
        a = annot[j]
        # select list of dict of tokens w/ annnotations and add column w/ no. of words to each dict:
        b = a['spans']
        # add noWords to b dict. note: b is list of dicts w/ annotations; tokens not on this list don't have annotations
        if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
            #print(b)
            for i in range(0,len(annot[j]['tokens'])):
                    # now break-up label into 1st occurrence (leading "B-") and subsequent occurrences (leading "I-") (only for non "O"'s)
                    if annot[j]['tokens'][i]['label'] != "O":
                        if i==0:
                            annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label']
                        else: 
                            if annot[j]['tokens'][i]['label'] == annot[j]['tokens'][i-1]['label'][2:]: # need to remove the leading "B-" that we had already been added to c[i-1]
                                annot[j]['tokens'][i]['label'] = "I-" + annot[j]['tokens'][i]['label']
                            else:
                                annot[j]['tokens'][i]['label'] = "B-" + annot[j]['tokens'][i]['label'] 

In [84]:
# now get data into format that TorchRNN expects:
X=[] 
y=[]
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    auxX = []
    auxy = []
    if annot[j]['spans']!=[]: # are there annot for this example?
        for i in range(0,len(a)):
            #token_element = (a[i]['text'],a[i]['label'])
            auxX.append(a[i]['text'])
            auxy.append(a[i]['label'])
        X.append(auxX)
        y.append(auxy)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_test, y_train, y_test = X[:120], X[120:], y[:120], y[120:]
vocab = sorted({w for seq in X_train for w in seq}) + ["$UNK"]

In [89]:
print(X_train[0])

['DORNBIRN', 'In', 'der', 'Schulgasse', 'in', 'Dornbirn', 'hat', 'eine', '71,93', 'Quadratmeter', 'große', 'Wohnung', 'für', 'einen', 'Quadratmeterpreis', 'von', '5533,71', 'Euro', 'den', 'Besitzer', 'gewechselt', '.', 'Dieser', 'beinhaltet', 'auch', 'einen', 'Pkw-Abstellplatz', '.', 'Käufer', 'der', 'Wohnung', 'mit', '9,86', 'Quadratmetern', 'Terrasse', 'ist', 'die', 'ValLiLean', 'Beteiligungs-', 'und', 'Immobilienverwaltungs', 'GmbH', 'Beim', 'Verkäufer', 'handelt', 'es', 'sich', 'um', 'die', 'Karrenblick', 'Projekt', 'GmbH', ' ', 'Der', 'Kaufpreis', 'liegt', 'bei', '398.040', 'Euro', '.', 'Unterzeichnet', 'wurde', 'der', 'Kaufvertrag', 'am', '18.', 'September', '.', 'Die', 'Verbücherung', 'datiert', 'mit', 'Oktober', '2020', '.', '.', '.']


In [85]:
%time _ = seq_mod.fit(X_train, y_train)
y_pred = seq_mod.predict(X_test)
print(y_test[0])
print(y_pred[0])

Stopping after epoch 35. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.249159574508667

Wall time: 1.78 s
['B-ORT', 'O', 'O', 'B-STRASSE', 'I-STRASSE', 'O', 'B-ORT', 'O', 'O', 'B-FLAECHE', 'O', 'O', 'B-IMMO_TYP', 'O', 'O', 'O', 'O', 'B-QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'B-GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DATUM_VERTRAG', 'I-DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'B-DATUM_VERBUECHERUNG', 'I-DATUM_VERBUECHERUNG', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [90]:
labels=seq_mod.classes_
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

In [91]:
# unfold all our data - NOTE: this means we don't care about per sentence results. 
# i.e. each classification is worth same regardless of sentence in which it occurs
y_test_unfold = [y for element in y_test for y in element]
y_pred_unfold = [y for element in y_pred for y in element]

In [92]:
print(classification_report(
    y_test_unfold, y_pred_unfold, labels=sorted_labels, digits=3
))

                       precision    recall  f1-score   support

                    O      0.773     0.988     0.867       643
B-DATUM_VERBUECHERUNG      0.000     0.000     0.000        13
I-DATUM_VERBUECHERUNG      0.000     0.000     0.000        12
      B-DATUM_VERTRAG      0.000     0.000     0.000        13
      I-DATUM_VERTRAG      0.000     0.000     0.000        14
            B-FLAECHE      0.000     0.000     0.000        15
            I-FLAECHE      0.000     0.000     0.000         0
        B-GESAMTPREIS      0.000     0.000     0.000        11
        I-GESAMTPREIS      0.000     0.000     0.000         0
           B-IMMO_TYP      0.000     0.000     0.000        19
           I-IMMO_TYP      0.000     0.000     0.000         0
            B-KAEUFER      0.000     0.000     0.000        10
            I-KAEUFER      0.000     0.000     0.000         8
                B-ORT      0.300     0.115     0.167        26
            B-QMPREIS      0.000     0.000     0.000  

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Remove "B-" and "I-" (in case they are present in labels)

In [119]:
for j in range(0,len(annot)):
    a = annot[j]
    b = a['spans']
    if b!=[]: #i.e. only try to add annotations to tokens if there are annotations to begin with
        for i in range(0,len(annot[j]['tokens'])):
                if annot[j]['tokens'][i]['label'] != "O":
                    if annot[j]['tokens'][i]['label'][:2]=="B-" or annot[j]['tokens'][i]['label'][:2]=="I-":
                        annot[j]['tokens'][i]['label']=annot[j]['tokens'][i]['label'][2:]

Try bi-directional LSTM

In [149]:
seq_mod = TorchRNNSequenceLabeler(
    vocab,
    early_stopping=True,
    eta=0.001,
    bidirectional=True)

In [132]:
print(y_train[0])

['ORT', 'O', 'O', 'STRASSE', 'O', 'ORT', 'O', 'O', 'FLAECHE', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'O', 'O', 'QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'TERRASSENGROESSE', 'O', 'O', 'O', 'O', 'KAEUFER', 'KAEUFER', 'KAEUFER', 'KAEUFER', 'KAEUFER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VERKAEUFER', 'VERKAEUFER', 'VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERTRAG', 'DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O', 'O', 'O']


In [150]:
%time _ = seq_mod.fit(X_train, y_train)

None


Stopping after epoch 18. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 2.1157665252685547

Wall time: 2.12 s


In [123]:
y_pred = seq_mod.predict(X_test)
print(y_test[0])
print(y_pred[0])

labels=seq_mod.classes_
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

# unfold all our data - NOTE: this means we don't care about per sentence results. 
# i.e. each classification is worth same regardless of sentence in which it occurs
y_test_unfold = [y for element in y_test for y in element]
y_pred_unfold = [y for element in y_pred for y in element]

print(classification_report(
    y_test_unfold, y_pred_unfold, labels=sorted_labels, digits=3
))

['ORT', 'O', 'O', 'STRASSE', 'STRASSE', 'O', 'ORT', 'O', 'O', 'FLAECHE', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'O', 'O', 'QMPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'VERKAEUFER', 'O', 'O', 'O', 'O', 'O', 'GESAMTPREIS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERTRAG', 'DATUM_VERTRAG', 'O', 'O', 'O', 'O', 'O', 'DATUM_VERBUECHERUNG', 'DATUM_VERBUECHERUNG', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'IMMO_TYP', 'O', 'O', 'TERRASSENGROESSE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'TERRASSENGROESSE', 'TERRASSENGROESSE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
                     precision    recall  f1-score   support

                  O      0.760     0.893     0.821       643
            KAEUFER      0.000     0.000     0.000        18
DATUM_VERBUECHERUN